# Submission 5


In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

import json
import random

Loads the training data

In [2]:
data = json.loads(open('data/train.json').read())

#   replacing spaces with underlines, make into strings for count vectorizer
X_full = ([[ingredient.replace(' ', '_') for ingredient in food['ingredients']]\
      for food in data])

#  ^Look at the badass list comprehension right there
X_full = [' '.join(ingredients) for ingredients in X_full]

y_full = map(lambda x: x['cuisine'], data)

X_full, y_full = np.asarray(X_full), np.asarray(y_full)

assert len(X_full) == len(y_full)

Loads the test data

In [3]:
test_data = json.loads(open('data/test.json').read())

X_out_test = ([[ingredient.replace(' ', '_').encode('ascii', 'ignore')\
                for ingredient in food['ingredients']]\
                for food in test_data])
X_out_test = [' '.join(ingredients) for ingredients in X_out_test]
X_out_test = np.asarray(X_out_test)

In [4]:
print '\n\n'.join(X_out_test[:5])

baking_powder eggs all-purpose_flour raisins milk white_sugar

sugar egg_yolks corn_starch cream_of_tartar bananas vanilla_wafers milk vanilla_extract toasted_pecans egg_whites light_rum

sausage_links fennel_bulb fronds olive_oil cuban_peppers onions

meat_cuts file_powder smoked_sausage okra shrimp andouille_sausage water paprika hot_sauce garlic_cloves browning lump_crab_meat vegetable_oil all-purpose_flour freshly_ground_pepper flat_leaf_parsley boneless_chicken_skinless_thigh dried_thyme white_rice yellow_onion ham

ground_black_pepper salt sausage_casings leeks parmigiano_reggiano_cheese cornmeal water extra-virgin_olive_oil


Maybe use XGBoost?

In [5]:
trans_clf = Pipeline([('count_vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('k_best', SelectKBest(k=5200)),
                      ('clf', SGDClassifier())])

In [6]:
trans_clf.fit(X_full, y_full)

Pipeline(steps=[('count_vect', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
    ...   penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False))])

In [7]:
preds = trans_clf.predict(X_full)
np.mean(preds == y_full)

0.82305023382108911

In [8]:
df = pd.DataFrame(columns=['id','cuisine'])
df['id'] = map(lambda x: x['id'], test_data)

preds = trans_clf.predict(X_out_test)
df['cuisine'] = preds
df.index=df.id
df = df.drop('id', axis=1)
df.head()

,cuisine
id,
18009,irish
28583,southern_us
41580,italian
29752,cajun_creole
35687,italian


In [14]:
# df.to_csv('submissions/submission_5.csv')